In [1]:
# Great Lakes SST Preprocessing and Visualization Notebook

import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from deepsensor.data import DataProcessor

# Import custom modules
from deepsensor_greatlakes.dataset_loader import (
    load_metadata, 
    load_glsea_data_combined, 
    load_context_data
)
from deepsensor_greatlakes.preprocessor import SeasonalCycleProcessor

In [ ]:
# Load project metadata
metadata = load_metadata()

# Load context data
context_data = load_context_data(metadata)

# Load GLSEA data for specific years
glsea_ds = load_glsea_data_combined(metadata, years=[1995, 1996])

# Calculate seasonal cycle
seasonal_processor = SeasonalCycleProcessor()
seasonal_processor.calculate(glsea_ds)

# Compute anomalies
sst_anomalies = seasonal_processor.compute_anomalies(glsea_ds)

# Initialize DeepSensor DataProcessor
data_processor = DataProcessor(x1_name="lat", x2_name="lon")

# Normalize the anomalies
sst_normalized = data_processor(sst_anomalies)


Loading context data:
Loading bathymetry

Bathymetry Validation:
Dimensions: 240 x 360
Depth range: 10.00m to 316.63m
Coverage: 23.0%
Loading lakemask

Lakemask Validation:
Dimensions: 838 x 1181
Values: [0. 1.]
Coverage: 100.0%
Loading glsea for 1995
Loading glsea for 1996

Combining GLSEA datasets...

GLSEA Data Validation:
Time range: 1995-01-01T00:00:00.000000000 to 1996-12-31T00:00:00.000000000
Spatial dimensions: 838 x 1181
SST range: 0.00°C to 29.49°C
Water coverage: 14.1%


In [ ]:
# Visualization
plt.figure(figsize=(15, 10))
ax = plt.axes(projection=ccrs.PlateCarree())

# Add map features
ax.add_feature(cfeature.LAND, facecolor='lightgrey')
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.LAKES, alpha=0.5)
ax.add_feature(cfeature.RIVERS)

# Calculate mean anomaly
sst_mean = sst_normalized.sst.mean(dim='time')

# Plot the SST anomaly data
im = ax.pcolormesh(sst_normalized.lon, sst_normalized.lat, sst_mean, 
                   transform=ccrs.PlateCarree(), 
                   cmap='RdBu_r',  # Diverging colormap
                   )

# Add a colorbar
plt.colorbar(im, ax=ax, label='Normalized SST Anomaly (°C)')

# Set the map extent to focus on the Great Lakes
ax.set_extent([-92.5, -75.5, 38.5, 51], crs=ccrs.PlateCarree())

# Add a title
plt.title('Mean Normalized Sea Surface Temperature Anomaly (1995-1996)')

plt.tight_layout()
plt.show()

In [ ]:
# Optional: Save processors and seasonal cycle
#seasonal_processor.save("data/seasonal_cycles/glsea_1995_1996")
#data_processor.save("data/processors/glsea_sst_anomalies")

# Print some basic information
print("Data preprocessing and visualization complete.")
print(f"Normalized SST Anomaly Shape: {sst_normalized.sst.shape}")
print(f"Latitude Range: {sst_normalized.lat.min().values} to {sst_normalized.lat.max().values}")
print(f"Longitude Range: {sst_normalized.lon.min().values} to {sst_normalized.lon.max().values}")